
# Reef Clustering: Data generation and MCMC

This notebook shows how to simulate data from a Dirichlet
mixture model and run the MCMC algorithm in the paper *A Bayesian latent allocation model for clustering compositional data with application to the Great Barrier Reef* by Piancastelli, Friel, Vercelloni, Mengersen and Mira. These steps are implemented in Python to leverage of the `numba` high performance compiler. 
Results are then stored and read into `R` for post-processing via the `label.switching` library and visualization.

In [1]:
#Import Python libraries
import os
import scipy.stats as stats
import numpy 
import math
import time
from numba import jit
import random
import pandas as pd
from sklearn import preprocessing
from sklearn.cluster import KMeans
import dirichlet
import pickle
import multiprocessing as mp

In [2]:
# Read in Python scripts with the functions
from MH_z import *
from MH_rho import *
from Main import *

## 1. Simulating from a Dirichlet Mixture

After loading the necessary libraries and scripts, we simulate 400 observations from a 4-dimensional Dirichlet mixture with two components and parameters (5,5,5,5) and (5, 5,1,1) in the next code chunk. The main functions are those in `Main.py` for which a brief documentation is provided. We can access this by calling `help` on them. For example:

In [4]:
help(sim_dirichlet_mixture)

Help on function sim_dirichlet_mixture in module Main:

sim_dirichlet_mixture(rho, n)
    Simulates data from a Dirichlet mixture model. 
    
    rho: numpy array where the number of rows is the number of mixture components
    n: sample size
    
    Output: dictionary. 'data' is the simulated compositional data, 'z' are the cluster allocations.



In [7]:
#Simulate some data
rho_true = numpy.array([[5, 5, 5, 5],
                        [5, 5, 1, 1]])
sim = sim_dirichlet_mixture(rho_true, 400)
p = sim['data']

Cluster allocations are simulated at random and can be obtained by accessing `z` in the `sim` dictionary. Alternatively, we can read in the reef data sets from `Data\`.

## 2. Running MCMC

To run `m` MCMC chains, we create a list of this length containing initial parameter values for each chain. This can be done using the function `initial_values`.

In [12]:
m = 2 #Number of MCMC chains 
k = 2 #Number of clusters to fit the model

inits = initial_values(p, k, m)

We are now ready to parallelize our main function `MCMC_dirichlet` over the initial values list. In the function's description we can check the required arguments and expected output. We set the chain configurations and
hyperparameters in the next code chunk. The default values of `perv_var` and `sigma_a` of 0.7 and 0.5 will be used in this example.

In [13]:
burn_in = 100000
thin = 200
chain_output = 1000

hyperparams_dict = {'delta': 0.5,
                   'gamma': 0.2,
                   'phi': 5.0,
                   'lambda': 6.0}

In the next chunk, we refer to the `multiprocessing` library to run the `m` chains in parallel. It can take some minutes depending on how many iterations are required but progress information will be printed on screen.

In [14]:
# Parallellize over the length of inits list
n_cores = 2 # Set the number of cores 

pool = mp.Pool(n_cores)  # Number of processors to use
results = []
results = pool.starmap_async( MCMC_dirichlet, [(i, inits, p, burn_in,
                                      thin, chain_output, hyperparams_dict) for i in range(m)]).get()
pool.close()

Burn in step:  10000
Burn in step:  10000
Burn in step:  20000
Burn in step:  20000
Burn in step:  30000
Burn in step:  30000
Burn in step:  40000
Burn in step:  40000
Burn in step:  50000
Burn in step:  50000
Burn in step:  60000
Burn in step:  60000
Burn in step:  70000
Burn in step:  70000
Burn in step:  80000
Burn in step:  80000
Burn in step:  90000
Burn in step:  90000
Burn in step:  100000
Step:  0
Burn in step:  100000
Step:  0
Step:  10000
Step:  10000
Step:  20000
Step:  20000
Step:  30000
Step:  30000
Step:  40000
Step:  40000
Step:  50000
Step:  50000
Step:  60000
Step:  60000
Step:  70000
Step:  70000
Step:  80000
Step:  80000
Step:  90000
Step:  90000
Step:  100000
Step:  100000
Step:  110000
Step:  110000
Step:  120000
Step:  120000
Step:  130000
Step:  130000
Step:  140000
Step:  140000
Step:  150000
Step:  150000
Step:  160000
Step:  160000
Step:  170000
Step:  170000
Step:  180000
Step:  180000
Step:  190000
Step:  190000


Results can be saved at read into R for post-processing.

In [11]:
file_name ="Example results/MCMC_example.p"
pickle.dump(results,open(file_name,'wb'), protocol=2)     